## Try this Notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truefoundry/mlfoundry-examples/blob/main/raw-examples/xgboost/titanic_survival_classification.ipynb)

## Install dependencies

In [ ]:
! pip install --quiet "numpy>=1.0.0,<2.0.0" "pandas>=1.0.0,<2.0.0" "matplotlib>=3.5.2,<3.6.0" "seaborn>=0.11.0,<0.12.0" "xgboost>=1.6.1,<1.7.0" scikit-learn shap==0.40.0

## Titanic Survival Classification

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import shap

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

### Load data and process it
- Impute null and missing values
- Map categorical values to numerical values

In [ ]:
titanic = sns.load_dataset('titanic')
titanic.info()

In [ ]:
titanic.head()

In [ ]:
titanic.drop(['who', 'embark_town', 'alive'], inplace=True, axis=1)

In [ ]:
titanic.isnull().sum()

In [ ]:
pd.value_counts(titanic['embarked'])

In [ ]:
pd.value_counts(titanic['deck'])

In [ ]:
titanic['embarked'].fillna('S', inplace=True)
titanic['deck'].fillna('C', inplace=True)
titanic['age'].fillna(titanic['age'].median(), inplace=True)

In [ ]:
titanic.isnull().values.any()

In [ ]:
titanic.head()

In [ ]:
label_encoders = {}
for column in ['sex', 'embarked', 'class', 'adult_male', 'deck', 'alone']:
    label_encoders[column] = LabelEncoder()
    titanic[column] = label_encoders[column].fit_transform(titanic[column])

In [ ]:
X = titanic.copy().drop(['survived'], axis=1)
y = titanic['survived']

### Split Dataset into Training and Validation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

### Training model

In [ ]:
gbm = xgb.XGBClassifier(n_estimators=20, max_depth=5)
gbm.fit(X_train, y_train)
print('Accuracy:', gbm.score(X_test, y_test))

In [ ]:
print(gbm.get_params())

In [ ]:
import pickle

with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

### Computing predictions

In [ ]:
y_pred_train = gbm.predict(X_train)
y_pred_test = gbm.predict(X_test)

### Computing metrics

In [ ]:
metrics = {
    'train/accuracy': accuracy_score(y_train, y_pred_train),
    'train/precision': precision_score(y_train, y_pred_train),
    'train/recall': recall_score(y_train, y_pred_train),
    'train/f1': f1_score(y_train, y_pred_train),
    'test/accuracy': accuracy_score(y_test, y_pred_test),
    'test/precision': precision_score(y_test, y_pred_test),
    'test/recall': recall_score(y_test, y_pred_test),
    'test/f1': f1_score(y_test, y_pred_test),
}
print(metrics)